In [70]:
from nsepython import *
import pandas as pd
from datetime import datetime
import os

In [20]:
from crewai.tools import tool

In [71]:
os.environ["OPENAI_API_KEY"] = "Add_your_key"
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"

In [22]:
###Tool to fetch current price of a stock##############
@tool("get_current_price")
def get_current_price(stock_symbol:str, series:str)->str:
    """Fetches the current price of the stock"""
    current_date = datetime.now().strftime("%d-%m-%Y")
    result=equity_history(stock_symbol,series,start_date=current_date,end_date=current_date)
    result.drop(columns=['_id','CH_MARKET_TYPE','CH_SERIES','TIMESTAMP','CH_TOTAL_TRADES','CH_ISIN','createdAt','updatedAt','SLBMH_TOT_VAL','VWAP','mTIMESTAMP','__v'])
    result = result.to_json(orient='records')
    
    return result

In [23]:
####function to replace col names with proper descriptions#############
key_mapping = {
    're_to_dt': 'End Date of Reporting Period',
    're_from_dt': 'Start Date of Reporting Period',
    're_create_dt': 'Date of Result Reporting/Publication',
    're_res_type': 'Type of Result',
    're_excepn_items_new': 'Exceptional Items During Reporting Period',
    're_desc_note_seg': 'Segment-Specific Notes/Comments',
    're_staff_cost': 'Employee/Staff-Related Expenses',
    're_tot_exp_exc_pro_cont': 'Total Expenses Excluding Provisions/Contingencies',
    're_grs_npa': 'Gross Non-Performing Assets (NPA)',
    're_int_expd': 'Interest Expenditure',
    're_con_pro_loss': 'Consolidated Profit/Loss for the Period',
    're_net_sale': 'Net Sales Revenue',
    're_oth_exp': 'Other Expenses',
    're_rawmat_consump': 'Raw Material Consumption',
    're_diluted_eps': 'Diluted Earnings Per Share (EPS)',
    're_inc_dre_sttr': 'Incremental Deferred Tax Asset/Liability',
    're_pur_trd_goods': 'Purchases of Traded Goods',
    're_face_val': 'Face Value of Equity Shares',
    're_income_inv': 'Income from Investments',
    're_deff_tax': 'Deferred Tax',
    're_share_associate': 'Share of Profit/Loss from Associates',
    're_ret_asset': 'Return on Assets/Retired Assets',
    're_proloss_ord_act': 'Profit/Loss from Ordinary Activities',
    're_notes_to_ac': 'Notes Accompanying Accounts',
    're_grs_npa_per': 'Percentage of Gross NPAs',
    're_excepn_items': 'Exceptional Items',
    're_int_new': 'New Interest Income/Expenditure',
    're_pro_los_frm_dis_opr': 'Profit/Loss from Discontinued Operations',
    're_goi_per_shhd': 'Government Shareholding Percentage',
    're_basic_eps_for_cont_dic_opr': 'Basic Earnings Per Share (EPS) for Continuing/Discontinued Operations',
    're_oth_oper_exp': 'Other Operating Expenses',
    're_remarks': 'General Remarks/Notes',
    're_seg_remarks': 'Segment-Wise Remarks',
    're_dilut_eps_for_cont_dic_opr': 'Diluted EPS for Continuing/Discontinued Operations',
    're_desc_note_fin': 'Notes/Descriptions About Financial Data',
    're_per_grs_npa': 'Percentage of Gross NPAs',
    're_pro_loss_bef_tax': 'Profit/Loss Before Tax',
    're_res_reval': 'Revaluation of Reserves',
    're_int_ser_cov': 'Interest Service Coverage Ratio',
    're_cap_ade_rat': 'Capital Adequacy Ratio',
    're_debt_eqt_rat': 'Debt-Equity Ratio',
    're_curr_tax': 'Current Tax Liabilities',
    're_net_profit': 'Net Profit for the Period',
    're_total_inc': 'Total Income',
    're_oth_inc_new': 'Other New Income Sources',
    're_oth_tot_exp': 'Total of Other Expenses',
    're_seq_num': 'Sequence Number (Internal Identifier)'
}
def replace_keys(data, mapping):
    updated_data = []
    for obj in data:
        new_obj = {mapping.get(key, key): value for key, value in obj.items() if key in mapping}
        updated_data.append(new_obj)
    return updated_data


In [30]:
########Tool to get Latest news of a company##################
from bs4 import BeautifulSoup, Comment
@tool("get_recent_stock_news")
def get_recent_stock_news(stock_name:str, num_news:int)->list:
    """Gets the latest news and updates of a company."""
    # Format the stock name for the search URL (replace spaces with hyphens)
    stock_name_formatted = stock_name.replace(" ", "-")
    url = f"https://www.moneycontrol.com/news/tags/{stock_name_formatted}.html"

    try:
        # Send an HTTP request to fetch the webpage
        response = requests.get(url)
        response.raise_for_status()  # Check for HTTP errors

        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the news items (update selector based on website structure)
        news_items = soup.find_all('li', class_='clearfix', limit=num_news)
        final_news =[]

        if news_items:
            news_list = []
            for news in news_items:
                headline = news.find('h2').text.strip() if news.find('h2') else 'No headline found'
                link = news.find('a')['href'] if news.find('a') else 'No link available'

                # Extract date from comments
                date_comment = news.find(string=lambda text: isinstance(text, Comment))
                date_text = date_comment.strip() if date_comment else 'No date available'

                news_list.append({"headline": headline, "link": link, "date": date_text})
            
            for idx, news in enumerate(news_list, start=1):
                news_item = {}
                news_item['headline'] = news['headline']
                news_item['Date'] = news['date']
                final_news.append(news_item)

            
            return final_news
        else:
            return {"error": "No news found for the stock."}

    except requests.exceptions.RequestException as e:
        return {"error": f"Request failed: {e}"}

In [95]:
#########Tool to get financial reports of a company #################
# @tool("get_financials")
# def get_financials(stock_symbol:str)->list:
#     """Gets the financials reports of company for the last 5 quarters"""
#     fin_res = nse_past_results(stock_symbol)
#     res= replace_keys(fin_res['resCmpData'],key_mapping)
#     return res

In [96]:
@tool("get_financials")
def get_financials(stock_symbol: str) -> list:
    """Gets the financials reports of company for the last 5 quarters"""
    try:
        # Retrieve data
        fin_res = nse_past_results(stock_symbol)
        if not fin_res:
            raise ValueError(f"nse_past_results returned None for stock symbol: {stock_symbol}")
        
        # Validate key
        if 'resCmpData' not in fin_res or fin_res['resCmpData'] is None:
            raise KeyError(f"'resCmpData' key missing or None in nse_past_results response: {fin_res}")

        # Process and return results
        res = replace_keys(fin_res['resCmpData'], key_mapping)
        return res

    except Exception as e:
        # Log and re-raise the exception for further analysis
        raise RuntimeError(f"Error in get_financials: {str(e)}")


In [27]:
########Tool to get market research of a company##################

from openai import OpenAI

client = OpenAI(api_key="Add_your_key")
@tool("get_market_research_report")
def get_market_research_report(stock_symbol:str)->str:
    """Gets the market research of the sector to which the company belongs"""
    prompt = f""" You are a market research agent responsible for analyzing the sector of the stock - {stock_symbol}. Your tasks include:
                  Identifying the sector and industry the stock belongs to and analyzing recent trends and developments within that sector.
                  Assessing the short-term (6-12 months) and long-term (3-5 years) growth potential of the sector, with a primary focus on its performance and opportunities in India, followed by an analysis of global trends and their impact on the sector.
                  Focusing your analysis on the following key factors, especially in the Indian context:
                  Market size and demand drivers
                  Competitive landscape and emerging players
                  Technological advancements and their sectoral adoption
                  Identifying the critical points that influence the given stock's potential based on the sector's performance and trends.
                  Finally, provide your findings as a short and concise summary that highlights only the key points influencing the stock. Avoid lengthy explanations and focus on actionable insights and crucial trends."""

    completion = client.chat.completions.create(
        model="gpt-4o",
        store=True,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return completion.choices[0].message.content

# stock_symbol = "RELIANCE"
# report = get_market_research(stock_symbol)
# print(report)


In [28]:
from crewai import Agent, Task
from typing import Any, Dict, List, Optional

In [58]:
financial_agent = Agent(
  role='Financial Data Aggregator',
  goal=' Retrieve and compile comprehensive financial data for the company, including real-time stock prices, key performance metrics, and financial reports (e.g., annual and quarterly).',
  backstory=' A seasoned expert adept at extracting and organizing all relevant financial details for thorough analysis.',
  tools=[get_current_price,get_financials],
  memory=True, # Enable memory
  verbose=True,
  max_rpm=None, # No limit on requests per minute
  max_iter=10, # Default value for maximum iterations
  llm ="gpt-4o"
)

In [59]:
market_research_agent = Agent(
  role='Sector Market Analyst',
  goal='Conduct in-depth market research to analyze industry trends, competition, and key market dynamics affecting the company’s sector',
  backstory=' A specialist in market analysis, skilled at identifying sector trends, competitor activities, and emerging opportunities to inform strategic decisions.',
  tools=[get_market_research_report],
  memory=True, # Enable memory
  verbose=True,
  max_rpm=None, # No limit on requests per minute
  max_iter=10, # Default value for maximum iterations
   llm ="gpt-4o"
)

In [60]:
news_agent = Agent(
  role='News Aggregator',
  goal='Gather, filter, and provide recent news and updates about the company, focusing on events that impact stock performance or market perception.',
  backstory=' A proficient news-fetching agent skilled at sourcing relevant, actionable information from credible news outlets to aid in detailed company analysis.',
  tools=[get_recent_stock_news],
  memory=True, # Enable memory
  verbose=True,
  max_rpm=None, # No limit on requests per minute
  max_iter=10, # Default value for maximum iterations
   llm ="gpt-4o"
)

In [61]:
analysis_agent = Agent(
  role='Stock Analyst',
  goal='Integrate data from financial, market, and news agents to perform comprehensive analysis and deliver actionable Buy/Sell/Hold recommendations based on insights along with consolidated analysis of the stock',
  backstory='A strategic thinker proficient in combining multifaceted information to derive meaningful recommendations, balancing risk and opportunity in stock investments.',
  memory=True, # Enable memory
  verbose=True,
  max_rpm=None, # No limit on requests per minute
  max_iter=10, # Default value for maximum iterations,
   llm ="gpt-4o"
)

In [81]:
STOCK ="Cipla"

In [82]:
get_financial_task = Task(
    description=f""" your task is to provide comprehensive financial data for the specified company. You have access to the following tools to accomplish this task:
get_current_price: Fetches the current price of the stock. for the stock - {STOCK}
get_financials: Gets the financial reports of the company for the last 5 quarters.""",
expected_output="""The outcome of this task is a structured response summarizing the financial data of the specified company. This includes:
Current Stock Price: The latest available stock price of the company.
Financial Reports: A summary of the financial reports for the last 5 quarters, including key metrics such as revenue, profit, and other relevant financial indicators. """,
agent=financial_agent,
)

In [83]:
get_latest_news_task = Task(
    description=f"As a news reporting task your job is to provide up-to-date information about the specified company. You have access to web scraping tool that can fetch news with its date. For the stock - {STOCK}",
    expected_output="The outcome of this task is a structured response summarizing the latest news and updates about the specified company along with the date.",
    agent=news_agent,
)

In [84]:
market_research_task = Task(
    description=f"As a market research task your job is to provide comprehensive market research data for the specified sector. This includes analyzing market trends, identifying key players, and summarizing the competitive landscape. You have access to various tools and resources to accomplish this task. For the stock - {STOCK}",
    expected_output="""The outcome of this task is a structured response summarizing the market research data for the specified sector. This includes:
Market Trends: An overview of the current trends and developments in the sector.
Key Players: A list of major companies operating in the sector and their market positions.
Competitive Landscape: An analysis of the competitive environment, including strengths, weaknesses, opportunities, and threats (SWOT analysis).
Market Size and Growth: Information on the market size, growth rate, and potential for future expansion.""",
agent=market_research_agent,
)


In [85]:
analysis_task = Task(
    description="As an task responsible for performing a consolidated analysis of a given stock, your job is to provide a comprehensive evaluation based on the financial results, market research analysis, and the latest news from other agents. You will consider all relevant points and provide a recommendation on whether to buy, sell, or perform more analysis on the stock. Your analysis should be thorough and well-reasoned",
    expected_output=""" The outcome of this task is a structured response summarizing the consolidated analysis of the specified stock. This includes:
Financial Results: A summary of the company's financial performance, including key metrics such as revenue, profit, and earnings per share (EPS).
Market Research Analysis: An overview of the market trends, key players, competitive landscape, and market size and growth for the sector in which the company operates.
Latest News: A summary of the most recent news articles and updates related to the company, including any significant events or announcements.
Recommendation: A clear recommendation on whether to buy, sell, or perform more analysis on the stock, along with the reasons for the recommendation.""",
    agent=analysis_agent,
)

In [86]:
from crewai import Crew,Process

In [87]:
stock_crew = Crew(
  agents=[financial_agent,market_research_agent,news_agent,analysis_agent],
  tasks=[get_financial_task,get_latest_news_task,market_research_task,analysis_task],
  process=Process.sequential
)

2025-01-01 22:52:08,117 - 20548 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [100]:
result = stock_crew.kickoff()

# Agent: Financial Data Aggregator
## Task:  your task is to provide comprehensive financial data for the specified company. You have access to the following tools to accomplish this task:
get_current_price: Fetches the current price of the stock. for the stock - Cipla
get_financials: Gets the financial reports of the company for the last 5 quarters.


# Agent: Financial Data Aggregator
## Thought: To compile a comprehensive financial report, the first step is to gather the current stock price of Cipla.
## Using tool: get_current_price
## Tool Input: 
"{\"stock_symbol\": \"CIPLA\", \"series\": \"EQ\"}"
## Tool Output: 
[{"_id":"67754b92d3205bf7e5c3203e","CH_SYMBOL":"CIPLA","CH_SERIES":"EQ","CH_MARKET_TYPE":"N","CH_TIMESTAMP":"2025-01-01","TIMESTAMP":"2024-12-31T18:30:00.000Z","CH_TRADE_HIGH_PRICE":1547.85,"CH_TRADE_LOW_PRICE":1518.7,"CH_OPENING_PRICE":1531.95,"CH_CLOSING_PRICE":1529.1,"CH_LAST_TRADED_PRICE":1525,"CH_PREVIOUS_CLS_PRICE":1529,"CH_TOT_TRADED_QTY":1578715,"CH_TOT_TRADED_VA

In [101]:
TaskOutput = result
CrewOutput = result

In [102]:
TaskOutput

CrewOutput(raw="**Cipla Consolidated Stock Analysis**\n\n**Financial Results:**\nCipla, a leading pharmaceutical company, reported strong financial performance in its recent quarterly results. The company has consistently shown growth in key financial metrics:\n- **Revenue:** Cipla has seen a significant increase in revenue, driven by strong demand for generics and its expanding portfolio of chronic and acute therapeutics.\n- **Profit:** The company's profit margins have improved due to cost management and increased sales, especially in emerging markets.\n- **Earnings Per Share (EPS):** There has been a noticeable growth in EPS, reflecting the overall financial health and profitability of the company.\n\n**Market Research Analysis:**\n\n- **Sector and Industry Analysis:**\n  - **Sector:** Healthcare\n  - **Industry:** Pharmaceuticals\n\n- **Market Trends:**\n  1. The company is poised to benefit from increasing regulatory approvals for generic drugs both in India and globally, enhancin